## End of week Exercise - week 2



In [6]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

load_dotenv()

router = OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

In [7]:
system_prompt = """
You are a senior technical architect and AI engineer.

You help developers understand:

- system design
- APIs
- backend architecture
- microservices
- AI integrations

Explain clearly and practically.
Use real-world examples when helpful.
"""

In [8]:
def lookup_docs(query):
    docs = {
        "microservices": "Microservices split large apps into small independent services communicating via APIs.",
        "api": "An API allows systems to communicate using structured requests.",
        "streaming": "Streaming allows partial responses in real time instead of waiting for full completion."
    }
    
    for key in docs:
        if key in query.lower():
            return docs[key]
    
    return None

In [9]:
def chat(message, history, model):

    history = history or []

    messages = [{"role": "system", "content": system_prompt}]

    for msg in history:
        messages.append(msg)

    # TOOL USE
    tool_answer = lookup_docs(message)
    if tool_answer:
        return history + [
            {"role": "user", "content": message},
            {"role": "assistant", "content": f"📚 Tool:\n{tool_answer}"}
        ]

    messages.append({"role": "user", "content": message})

    stream = router.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )

    reply = ""
    updated_history = history + [
        {"role": "user", "content": message},
        {"role": "assistant", "content": ""}
    ]

    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        reply += delta
        updated_history[-1]["content"] = reply
        yield updated_history

In [10]:
demo = gr.ChatInterface(
    fn=chat,
    additional_inputs=gr.Dropdown(
        [
            "openai/gpt-4.1-mini",
            "anthropic/claude-3-haiku",
            "mistralai/mistral-7b-instruct"
        ],
        value="openai/gpt-4.1-mini",
        label="Choose Model"
    ),
    title="🧠 Technical AI Assistant"
)

demo.launch()

/Users/sirphiltechpaxe/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.


To Extended to sppech to text. could test as I am utilizing openRouter